In [ ]:
%pip install beautifulsoup4 requests pandas

In [ ]:

from bs4 import BeautifulSoup
from datetime import datetime, timezone
import pandas as pd
import requests
from urllib.parse import urlparse, parse_qs
from zoneinfo import ZoneInfo

# process each row of the annoucement table data
def extract_cells(date, price, link):
  return [
    datetime.strptime(date.get_text(' ', strip=True), r'%d/%m/%Y %I:%M %p').replace(tzinfo=ZoneInfo("Australia/Sydney")).astimezone(timezone.utc),
    price.img is not None,
    link.a.get_text().splitlines()[1].strip(),
    parse_qs(urlparse(link.a['href']).query)['idsId'][0] #/asx/v2/statistics/displayAnnouncement.do?display=pdf&idsId=1234
  ]

# request the announcements data for the symbol and year (default to current year)
def get_announcements(sym, year=datetime.today().year):
  res = requests.get(f"https://www.asx.com.au/asx/v2/statistics/announcements.do", params={
    'by': 'asxCode',
    'asxCode': sym,
    'timeframe': 'Y',
    'year': year
  })
  soup = BeautifulSoup(res.content, 'html.parser')
  return [extract_cells(*row.find_all('td')) for row in soup.select('announcement_data tbody > tr')]

# Get the current asx companies
asx_companies = pd.read_csv('../../../reference/asx/asx_companies.csv', converters={
    'Listing date': lambda x: datetime.strptime(x, '%d/%m/%Y').date(),
    'Market Cap': lambda x: int(x) if x.isdigit() else 0
  })

for sym in asx_companies.query("`GICs industry group` == 'Materials'").sort_values(by='Market Cap', ascending=False)['ASX code'].head(1):
  display(get_announcements(sym))

[[datetime.datetime(2026, 1, 19, 21, 56, tzinfo=datetime.timezone.utc),
  True,
  'Quarterly Activities Report',
  '03051180'],
 [datetime.datetime(2026, 1, 19, 21, 54, tzinfo=datetime.timezone.utc),
  False,
  'Jansen Stage 1 Potash Project Update',
  '03051179'],
 [datetime.datetime(2026, 1, 15, 0, 11, tzinfo=datetime.timezone.utc),
  False,
  'Notification of cessation of securities - BHP',
  '03050153'],
 [datetime.datetime(2026, 1, 13, 23, 9, tzinfo=datetime.timezone.utc),
  False,
  'Notification regarding unquoted securities - BHP',
  '03049719'],
 [datetime.datetime(2026, 1, 11, 23, 8, tzinfo=datetime.timezone.utc),
  False,
  'Notification regarding unquoted securities - BHP',
  '03048984']]